## Install prerequisites

In [27]:
from importlib.metadata import version
version('torch')

'2.4.0'

In [ ]:
pip install --upgrade transformers

https://docs.vllm.ai/en/latest/

In [12]:
version('transformers')

'4.44.2'

In [ ]:
pip install --upgrade vllm

In [3]:
version('vllm')

'0.6.1.post2'

https://sglang.readthedocs.io/en/latest/

In [ ]:
pip install --upgrade "sglang[all]"

In [ ]:
pip install flashinfer -i https://flashinfer.ai/whl/cu124/torch2.4/

In [7]:
version('sglang')

'0.3.1.post2'

In [28]:
version('flashinfer')

'0.1.6+cu124torch2.4'

https://github.com/ollama/ollama-python

https://github.com/ollama/ollama/tree/main/docs

In [ ]:
pip install --upgrade ollama

In [10]:
version('ollama')

'0.3.3'

## Test installation

In [1]:
test_messages = [
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages du Crédit Mutuel ?"}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages du Crédit Agricole ?"}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages de la Société Générale ?"}
],
[
    {"role": "system", "content": "Tu es un assistant utile et professionnel qui répond toujours en français."},
    {"role": "user", "content": "Quels sont les avantages de la BNP ?"}
]
]

In [2]:
test_models = {
    "llama-3.1" : "meta-llama/Meta-Llama-3.1-8B-Instruct"
}

In [3]:
from transformers import AutoTokenizer

def format_prompt(messages, model):
    tokenizer = AutoTokenizer.from_pretrained(model)
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

In [4]:
format_prompt(test_messages, test_models["llama-3.1"])

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nTu es un assistant utile et professionnel qui répond toujours en français.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuels sont les avantages du Crédit Mutuel ?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n',
 '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nTu es un assistant utile et professionnel qui répond toujours en français.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuels sont les avantages du Crédit Agricole ?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n',
 '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 Jul 2024\n\nTu es un assistant utile et professionnel qui répond toujours en français.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nQuels sont 

### vLLM

In [5]:
# Authenticate VLLM with Huggingface Hub
import os

with open("/workspace/hftoken", 'r') as file:
    myhftoken = file.read().strip()

os.environ["HF_TOKEN"]=myhftoken

In [6]:
import time
from vllm import LLM, SamplingParams

def vllm_load(model):    
    llm = LLM(model, gpu_memory_utilization=0.99, max_model_len=8192)
    llm._model = model
    return llm

def vllm_generate(messages, llm):    
    print(f"vLLM performance test:")
    
    prompts = format_prompt(messages, llm._model)
    sampling_params = SamplingParams(temperature=0.7, top_p=0.8, repetition_penalty=1.05, max_tokens=512)
    # warmup
    outputs = llm.generate(prompts[0], sampling_params)
    print(f"Generated text: {outputs[0].outputs[0].text!r}")
    
    for batch_size in range(1, len(messages) + 1):
        start_time = time.time()  # Record the start time
        outputs = llm.generate(prompts[0:batch_size], sampling_params)
        end_time = time.time()  # Record the end time
            
        # Print the outputs.
        tokenscount = 0
        for output in outputs:
            generated_text = output.outputs[0].text
            tokenscount = tokenscount + len(output.outputs[0].token_ids)

        tokens_per_sec = tokenscount/(end_time-start_time)
        print(f"- batch size {batch_size}: {tokens_per_sec:.2f} tokens/sec ({batch_size} x {tokens_per_sec/batch_size:.2f})")

In [7]:
llm = vllm_load(test_models["llama-3.1"])

INFO 09-21 14:59:48 llm_engine.py:184] Initializing an LLM engine (v0.5.5) with config: model='meta-llama/Meta-Llama-3.1-8B-Instruct', speculative_config=None, tokenizer='meta-llama/Meta-Llama-3.1-8B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, collect_model_forward_time=False, collect_model_execute_time=False), seed=0, served_model_name=meta-llama/Meta-Llama-3.1-8B-Instruct, use_v2_block_manager=False, enable_prefix_caching=False)
WARNING 09-21 14:59:49 utils.py

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


INFO 09-21 14:59:52 model_runner.py:890] Loading model weights took 14.9888 GB
INFO 09-21 14:59:53 gpu_executor.py:121] # GPU blocks: 3610, # CPU blocks: 2048
INFO 09-21 14:59:54 model_runner.py:1181] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-21 14:59:54 model_runner.py:1185] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-21 15:00:05 model_runner.py:1300] Graph capturing finished in 11 secs.


In [8]:
vllm_generate(test_messages*8, llm)

vLLM performance test:


Processed prompts: 100%|█████████████| 1/1 [00:09<00:00,  9.43s/it, est. speed input: 6.68 toks/s, output: 54.32 toks/s]


Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre plusieurs avantages à ses membres et clients. Voici quelques-uns des principaux avantages :\n\n1. **Intérêts plus élevés sur les épargnes** : Le Crédit Mutuel propose des taux d'intérêt plus élevés sur les comptes d'épargne que de nombreux autres établissements bancaires.\n2. **Taux d'emprunt compétitifs** : Les prêts personnels, les prêts immobiliers et les prêts pour la mobilité sont proposés avec des taux d'intérêt attractifs.\n3. **Services personnalisés** : Le Crédit Mutuel offre des services personnalisés et adaptés aux besoins de ses membres et clients, grâce à une approche relationnelle et à une connaissance approfondie de leurs situations financières.\n4. **Sécurité et confidentialité** : Le Crédit Mutuel s'engage à protéger les données personnelles et financières de ses membres et clients, conformément aux règles de protection des données.\n5. **Participation aux décisions** : En tant que membre 

Processed prompts: 100%|█████████████| 1/1 [00:09<00:00,  9.29s/it, est. speed input: 6.78 toks/s, output: 55.13 toks/s]


- batch size 1: 55.11 tokens/sec (1 x 55.11)


...

- batch size 30: 1222.15 tokens/sec (30 x 40.74)


Processed prompts: 100%|███████| 31/31 [00:12<00:00,  2.47it/s, est. speed input: 155.98 toks/s, output: 1259.03 toks/s]


- batch size 31: 1258.56 tokens/sec (31 x 40.60)


Processed prompts: 100%|███████| 32/32 [00:12<00:00,  2.52it/s, est. speed input: 158.80 toks/s, output: 1288.18 toks/s]

- batch size 32: 1287.65 tokens/sec (32 x 40.24)


### SGLang

In [18]:
import json, time
import sglang

def sglang_load(model):
    runtime = sglang.Runtime(model_path=model)
    runtime._model = model
    return runtime

def sglang_generate(messages, runtime):
    print(f"SGLang performance test:")
    
    prompts = format_prompt(messages, runtime._model)
    sampling_params = { "temperature":0.7, "top_p":0.8, "repetition_penalty":1.05, "max_new_tokens":512 }
    # warmup
    output = json.loads(runtime.generate(prompt=prompts[0], sampling_params=sampling_params))
    print(f"Generated text: {output['text']!r}")
    
    for batch_size in range(1, len(messages) + 1):
        start_time = time.time()  # Record the start time
        outputs = json.loads(runtime.generate(prompt=prompts[0:batch_size], sampling_params=sampling_params))
        end_time = time.time()  # Record the end time
            
        # Print the outputs.
        tokenscount = 0
        for output in outputs:
            generated_text = output["text"]
            tokenscount = tokenscount + output["meta_info"]["completion_tokens"]

        tokens_per_sec = tokenscount/(end_time-start_time)
        print(f"- batch size {batch_size}: {tokens_per_sec:.2f} tokens/sec ({batch_size} x {tokens_per_sec/batch_size:.2f})")

In [6]:
runtime = sglang_load(test_models["llama-3.1"])

INFO 09-21 14:36:03 weight_utils.py:236] Using model weights format ['*.safetensors']


[rank0]:W0921 14:36:03.593000 140239112718016 torch/_inductor/compile_worker/subproc_pool.py:126] SubprocPool unclean exit
Traceback (most recent call last):
  File "/root/miniconda3/envs/wordslab-notebooks/lib/python3.11/site-packages/torch/_inductor/compile_worker/__main__.py", line 45, in <module>
    main()
  File "/root/miniconda3/envs/wordslab-notebooks/lib/python3.11/site-packages/torch/_inductor/compile_worker/__main__.py", line 38, in main
    pre_fork_setup()
  File "/root/miniconda3/envs/wordslab-notebooks/lib/python3.11/site-packages/torch/_inductor/async_compile.py", line 62, in pre_fork_setup
    from triton.compiler.compiler import triton_key
ImportError: cannot import name 'triton_key' from 'triton.compiler.compiler' (/workspace/wordslab-llms/.venv/lib/python3.11/site-packages/triton/compiler/compiler.py)


Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


In [21]:
sglang_generate(test_messages*8, runtime)

SGLang performance test:
Generated text: "Le Crédit Mutuel est une banque coopérative française qui offre divers avantages à ses membres, notamment :\n\n1. **Intérêts plus élevés sur les épargnes** : Le Crédit Mutuel propose des taux d'intérêt attractifs sur les comptes d'épargne, ce qui permet aux membres de gagner plus sur leurs économies.\n2. **Prêts à des conditions avantageuses** : Les prêts accordés par le Crédit Mutuel sont souvent moins chers que ceux proposés par les banques traditionnelles, avec des taux d'intérêt compétitifs et des conditions de remboursement flexibles.\n3. **Services personnalisés** : Le Crédit Mutuel offre des services personnalisés à ses membres, notamment en termes de conseil financier et de suivi de leur situation financière.\n4. **Participation aux décisions** : En tant que membre du Crédit Mutuel, vous avez la possibilité de participer aux décisions stratégiques de l'organisation et de voter pour les dirigeants.\n5. **Soutien à l'économie locale** : L

https://docs.vllm.ai/en/latest/quantization/fp8.html#quantization-process